In [ ]:
# Initial imports
from dateutil.parser import parse
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline
from sklearn import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import hvplot.pandas as hvplot

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image

In [ ]:
sq_aroon_df = pd.read_csv(Path('/work/Fintech-Project2/STRATEGY NOTEBOOKS/Daily Strategies/SQ DAILY AROON 25 TRADE STRATEGY.csv'), index_col = 'Date', infer_datetime_format= True)
daily_aroon = sq_aroon_df['aroon_signal'].to_frame()
daily_aroon.index = pd.to_datetime(daily_aroon.index, format = '%Y/%m/%d').strftime('%Y-%m-%d-%H:%M:%S')
daily_aroon.head()

,aroon_signal
Date,
2019-09-03-00:00:00,0
2019-09-04-00:00:00,0
2019-09-05-00:00:00,0
2019-09-06-00:00:00,0
2019-09-09-00:00:00,0


In [ ]:
sq_aroon_df = pd.read_csv(Path('/work/Fintech-Project2/STRATEGY NOTEBOOKS/Daily Strategies/SQ DAILY AROON 25 TRADE STRATEGY.csv'))
daily_aroon = sq_aroon_df[['Date','aroon_signal']]
daily_aroon["Date"] = daily_aroon["Date"].apply(lambda x : str(x) + ' 15:30:00')
daily_aroon.set_index('Date', inplace=True)
daily_aroon.head(1)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,aroon_signal
Date,
2019-09-03 15:30:00,0


In [ ]:
sq_rsi_df = pd.read_csv('/work/Fintech-Project2/STRATEGY NOTEBOOKS/Daily Strategies/SQ DAILY RSI TRADE STRATEGY.csv')
daily_rsi = sq_rsi_df[['rsi_signal', 'Date']]
daily_rsi['Date'] = daily_rsi["Date"].apply(lambda x : str(x) + ' 15:30:00')
daily_rsi.set_index('Date', inplace=True)
daily_rsi.head(1)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,rsi_signal
Date,
2019-09-03 15:30:00,0


# sq_macd_df

In [ ]:
sq_macd_df = pd.read_csv('/work/Fintech-Project2/TRADE_STRATS/SQ MACD TRADE STRATEGY.csv', infer_datetime_format = True, parse_dates=True)
sq_macd_df["Date"] = sq_macd_df["Unnamed: 0"].apply(lambda x : x[:-6])
#sq_macd_df['Date'] = sq_macd_df['Unnamed: 0'].apply(lambda x:parse(x))
sq_macd_df.drop("Unnamed: 0", axis=1, inplace = True)
sq_macd_df.set_index('Date', inplace=True)
sq_macd_df.head(1)

,Close,macd,signal,macd_signal,macd_position
Date,,,,,
2019-09-03 09:30:00,62.959999,0.0,0.0,0,1


In [ ]:
# # TINKERING

# sq_macd_df = pd.read_csv('/work/Fintech-Project2/TRADE_STRATS/SQ MACD TRADE STRATEGY.csv', infer_datetime_format = True, parse_dates=True)
# sq_macd_df["Date"] = sq_macd_df["Unnamed: 0"].apply(lambda x : x[:-6])
# sq_macd_df['Date'] = sq_macd_df['Unnamed: 0'].apply(lambda x:parse(x))
# sq_macd_df.drop("Unnamed: 0", axis=1, inplace = True)

# sq_macd_df.head(1)

In [ ]:
# sq_macd_df['Datetime'] = sq_macd_df['Date']
# sq_macd_df.head()

In [ ]:
# sq_macd_df.set_index('Date', inplace=True)
# sq_macd_df.index

In [ ]:
# sq_macd_df['Datetime'] = pd.to_datetime(sq_macd_df['Datetime'], utc = True)
sq_macd_df.index = pd.to_datetime(sq_macd_df.index, format = '%Y/%m/%d').strftime('%Y-%m-%d-%H:%M:%S')

In [ ]:
hourly = sq_macd_df.groupby([sq_macd_df.index.dt.year, sq_macd_df.index.dt.hour]).mean()
hourly.head()


AttributeError: 'Index' object has no attribute 'dt'

In [ ]:
sq_macd_df["Year"] = sq_macd_df["Date"].apply(lambda x : x.year)
sq_macd_df["Month"] = sq_macd_df["Date"].apply(lambda x : x.month)
sq_macd_df["Day"] = sq_macd_df["Date"].apply(lambda x : x.day)
sq_macd_df["Hour"] = sq_macd_df["Date"].apply(lambda x : x.hour)
sq_macd_df["Minute"] = sq_macd_df["Date"].apply(lambda x : x.minute)
hourly = sq_macd_df.groupby([sq_macd_df["Year"], sq_macd_df["Month"], sq_macd_df["Day"], sq_macd_df["Hour"]]).mean()
hourly.drop('Minute', 1, inplace=True)


KeyError: 'Date'

In [ ]:
# # TINKERING

# sq_macd_df["Year"] = sq_macd_df["Datetime"].apply(lambda x : x.year)
# sq_macd_df["Month"] = sq_macd_df["Datetime"].apply(lambda x : x.month)
# sq_macd_df["Day"] = sq_macd_df["Datetime"].apply(lambda x : x.day)
# sq_macd_df["Hour"] = sq_macd_df["Datetime"].apply(lambda x : x.hour)
# sq_macd_df["Minute"] = sq_macd_df["Datetime"].apply(lambda x : x.minute)
# hourly = sq_macd_df.groupby([sq_macd_df["Year"], sq_macd_df["Month"], sq_macd_df["Day"], sq_macd_df["Hour"]]).mean()
# hourly.drop('Minute', 1, inplace=True)




In [ ]:
hourly

NameError: name 'hourly' is not defined

In [ ]:
import datetime 
new_hourly = hourly.reset_index()
new_hourly['Date'] = new_hourly['Year'].astype(str) + '-' + new_hourly['Month'].astype(str) + '-' + new_hourly['Day'].astype(str) + ' ' + new_hourly['Hour'].astype(str)
new_hourly.index = new_hourly['Date']
new_hourly.drop(['Year', 'Month', 'Day', 'Hour', 'Date'], 1, inplace=True)
new_hourly.head()


NameError: name 'hourly' is not defined

In [ ]:
sq_rsi_df = pd.read_csv('/work/Fintech-Project2/STRATEGY NOTEBOOKS/Daily Strategies/SQ DAILY RSI TRADE STRATEGY.csv', infer_datetime_format= True, index_col= 'Date')
daily_rsi = sq_rsi_df['rsi_signal'].to_frame()
daily_rsi.index = pd.to_datetime(daily_rsi.index, format = '%Y/%m/%d').strftime('%Y-%m-%d-%H:%M:%S')
daily_rsi.tail()

,rsi_signal
Date,
2021-08-09-00:00:00,0
2021-08-10-00:00:00,0
2021-08-11-00:00:00,0
2021-08-12-00:00:00,0
2021-08-13-00:00:00,0


In [ ]:
combined_signals_df = pd.concat([daily_aroon, daily_rsi, sq_macd_df], axis = 1)
combined_signals_df

,aroon_signal,rsi_signal,Close,macd,signal,macd_signal,macd_position
Date,,,,,,,
2019-09-03 15:30:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-04 15:30:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-05 15:30:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-06 15:30:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-09 15:30:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2021-08-10-11:30:00,NaN,NaN,277.697998,1.711374,2.562172,0.0,0.0
2021-08-10-12:30:00,NaN,NaN,276.769989,1.388691,2.327476,0.0,0.0
2021-08-10-13:30:00,NaN,NaN,275.375000,1.008771,2.063735,0.0,0.0


In [ ]:
# pd.concat([sq_macd_df, daily_aroon, daily_rsi], axis=1).dropna()

NameError: name 'sq_macd_df' is not defined

In [ ]:
display(daily_aroon.index[0])
display(daily_rsi.index[0])
display(sq_macd_df.index[0])

'2019-09-03 00:00:00-00:00'

'2019-09-03-00:00:00'

'2019-09-03 09:30:00-04:00'

In [ ]:
display(daily_aroon.head(1))
display(daily_rsi.head(1))
display(sq_macd_df.head(1))

,aroon_signal
Date,
2019-09-03 00:00:00-00:00,0


,rsi_signal
Date,
2019-09-03-00:00:00,0


,Close,macd,signal,macd_signal,macd_position
2019-09-03 09:30:00-04:00,62.959999,0.0,0.0,0,1


In [ ]:
daily_aroon.reset_index(inplace = True)
daily_aroon

In [ ]:
daily_aroon['Date'].dt.date

AttributeError: Can only use .dt accessor with datetimelike values

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d12c285c-6789-4c45-9ad7-1be9b4d60622' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>